# 🔥 Smart Life Assistant — Multi-Agent Capstone (Vivek Parmar)
**Author:** Vivek Parmar (vish)
**Overview:** Multi-agent, rule-based Life Assistant demonstrating:
- Multi-agent system & sequential agents
- Custom tools (TimeOptimizer, LifeScore)
- Memory (session storage)
- Loop agent (daily checker)
- Evaluation & simple metrics

**How to run:** Run all cells. Edit `user_inputs` cell for personalization. No external APIs used.


In [14]:
# Basic imports
import datetime
from datetime import date, timedelta
import math
import random
import json
from pprint import pprint
import pandas as pd


In [15]:
# ============================
#   Smart Life Assistant (SLA)
# ============================

# ---------- Memory (simple persistent dict) ----------
class MemoryBank:
    def __init__(self):
        # simple in-memory storage; can be saved to file for persistence
        self.store = {"sessions": {}}

    def create_session(self, user_id="default"):
        if user_id not in self.store["sessions"]:
            self.store["sessions"][user_id] = {"history": [], "profile": {}}
        return user_id

    def write(self, user_id, key, value):
        self.create_session(user_id)
        self.store["sessions"][user_id].setdefault("profile", {})[key] = value

    def append_history(self, user_id, entry):
        self.create_session(user_id)
        self.store["sessions"][user_id]["history"].append({"time": datetime.datetime.now().isoformat(), "entry": entry})

    def read_profile(self, user_id, key):
        self.create_session(user_id)
        return self.store["sessions"][user_id]["profile"].get(key)

    def get_history(self, user_id):
        self.create_session(user_id)
        return self.store["sessions"][user_id]["history"]

    def export_json(self):
        return json.dumps(self.store, indent=2, default=str)

# ---------- Tools ----------
class TimeOptimizerTool:
    def analyze(self, waste_breakdown):
        # waste_breakdown: dict activity->minutes
        total_waste = sum(waste_breakdown.values())
        weekly_savings_est = round((total_waste/60) * 4, 1)
        suggestions = [
            "Use Pomodoro (25/5)",
            "Batch similar tasks",
            "Block social media to specific slots"
        ]
        return {"total_waste_min": total_waste, "weekly_hours_saved_est": weekly_savings_est, "suggestions": suggestions}

class LifeScoreTool:
    def compute(self, sleep_hrs, exercise_mins, study_mins, mood_rating):
        score = 40
        score += 10 if sleep_hrs>=7 else (5 if sleep_hrs>=6 else 0)
        score += 10 if exercise_mins>=45 else (5 if exercise_mins>=20 else 0)
        score += 10 if study_mins>=120 else (5 if study_mins>=60 else 0)
        score += int((mood_rating-5)*2)  # mood on 1-10
        return max(0, min(100, score))

# ---------- Agent Base ----------
class AgentBase:
    def __init__(self, memory: MemoryBank, tools: dict, user_id="default"):
        self.memory = memory
        self.tools = tools
        self.user_id = self.memory.create_session(user_id)

    def act(self, *args, **kwargs):
        raise NotImplementedError()

# ---------- Study Planner Agent ----------
class StudyAgent(AgentBase):
    def act(self, subject, exam_date_str, daily_hours=2):
        exam_date = datetime.datetime.strptime(exam_date_str, "%Y-%m-%d").date()
        today = date.today()
        days_left = max(0, (exam_date - today).days)
        if days_left <= 0:
            result = {"error": "Exam date must be in the future."}
            self.memory.append_history(self.user_id, {"study_attempt": result})
            return result

        topics = max(10, min(60, days_left*2))  # heuristic
        topics_list = [f"{subject} - Topic {i+1}" for i in range(topics)]
        per_day = max(1, math.ceil(len(topics_list)/days_left))
        plan = {}
        idx = 0
        for d in range(days_left):
            day = (today + timedelta(days=d)).isoformat()
            plan[day] = {"topics": topics_list[idx:idx+per_day], "hours": min(daily_hours, 6)}
            idx += per_day
            if idx>=len(topics_list):
                break
        out = {"subject": subject, "exam_date": str(exam_date), "days_left": days_left, "plan_preview": dict(list(plan.items())[:7]), "full_days": len(plan)}
        self.memory.append_history(self.user_id, {"study_generated": out})
        return out

# ---------- Fitness Agent ----------
class FitnessAgent(AgentBase):
    def act(self, height_cm, weight_kg, goal="build_muscle"):
        bmi = round(weight_kg/((height_cm/100)**2),2)
        calories = 2200 if goal=="build_muscle" else 1600
        plan = {
            "bmi": bmi,
            "calories": calories,
            "protein_g": int(1.8*weight_kg),
            "workouts": {
                "Mon":"Full body (home)",
                "Tue":"Cardio 25m",
                "Wed":"Legs + core",
                "Thu":"Active rest",
                "Fri":"Upper body",
                "Sat":"Core & mobility",
                "Sun":"Rest"
            }
        }
        self.memory.append_history(self.user_id, {"fitness_generated": plan})
        return plan

# ---------- Routine Agent (Sequential) ----------
class RoutineAgent(AgentBase):
    def act(self, wake_time="06:30", sleep_time="22:30", commitments=None):
        base = [
            ("Morning","Wake, freshen, 15m planning"),
            ("Focus 1","90m deep work/study"),
            ("Break","15m"),
            ("Focus 2","60m"),
            ("Workout","45-60m"),
            ("Evening Review","45-60m"),
            ("Wind down","No screens, relax"),
        ]
        if commitments:
            base.append(("Commitments", str(commitments)))
        out = {"wake":wake_time, "sleep":sleep_time, "blocks": base}
        self.memory.append_history(self.user_id, {"routine_generated": out})
        return out

# ---------- Motivation & Evaluation Agent ----------
class EvalAgent(AgentBase):
    def act(self, recent_metrics):
        # recent_metrics: dict with simple numbers
        messages = []
        if recent_metrics.get("study_minutes",0) < 60:
            messages.append("Try a shorter but consistent study session (25m).")
        if recent_metrics.get("exercise_mins",0) < 30:
            messages.append("Add a 20-minute walk or quick HIIT.")
        # sentiment style
        messages.append("Remember: Progress > Perfection. Take tiny steps daily.")
        self.memory.append_history(self.user_id, {"evaluation": {"metrics": recent_metrics, "advice": messages}})
        return {"advice": messages, "score_hint": random.choice(["Keep going!","Great consistency!","Push a little more today."])}

# ---------- Loop Agent: Daily Checker ----------
class DailyLoopAgent(AgentBase):
    def act(self, check_date=None):
        # This simulates a daily scheduled check (loop)
        today = (check_date or date.today()).isoformat()
        session = self.memory.store["sessions"].get(self.user_id, {})
        last = session.get("history", [])[-3:]  # last three events
        summary = {"date": today, "last_actions": last}
        # Evaluate life score using tools if data present in profile
        profile = session.get("profile", {})
        if profile.get("sleep_hours") and profile.get("exercise_mins"):
            score_tool = self.tools.get("life_score")
            summary["life_score"] = score_tool.compute(profile.get("sleep_hours",7), profile.get("exercise_mins",30), profile.get("study_minutes",60), profile.get("mood",6))
        self.memory.append_history(self.user_id, {"daily_check": summary})
        return summary

# ---------- Orchestrator: Sequential pipeline that demonstrates agents interacting ----------
class Orchestrator:
    def __init__(self, user_id="default"):
        self.memory = MemoryBank()
        self.tools = {"time_opt": TimeOptimizerTool(), "life_score": LifeScoreTool()}
        self.user_id = user_id
        # instantiate agents with same memory/tools to enable multi-agent interactions
        self.study_agent = StudyAgent(self.memory, self.tools, user_id)
        self.fitness_agent = FitnessAgent(self.memory, self.tools, user_id)
        self.routine_agent = RoutineAgent(self.memory, self.tools, user_id)
        self.eval_agent = EvalAgent(self.memory, self.tools, user_id)
        self.loop_agent = DailyLoopAgent(self.memory, self.tools, user_id)

    def run_pipeline(self, user_inputs):
        """
        Runs a sequential flow:
        1. Save profile to memory
        2. StudyAgent generates plan
        3. FitnessAgent generates plan
        4. RoutineAgent produces daily routine
        5. Tools analyze time waste & life score
        6. EvalAgent gives advice
        7. LoopAgent summarises
        Returns a combined report dict.
        """
        # save basic profile
        profile_keys = ["height_cm","weight_kg","sleep_hours","exercise_mins","study_minutes","mood","waste_breakdown"]
        for k in profile_keys:
            if k in user_inputs:
                self.memory.write(self.user_id, k, user_inputs[k])
        self.memory.append_history(self.user_id, {"pipeline_started": True})

        # 1 Study
        study_out = self.study_agent.act(user_inputs.get("subject","General"), user_inputs.get("exam_date",(date.today()+timedelta(days=20)).isoformat()), user_inputs.get("daily_hours",2))

        # 2 Fitness
        fitness_out = self.fitness_agent.act(user_inputs.get("height_cm",170), user_inputs.get("weight_kg",60), user_inputs.get("goal","build_muscle"))

        # 3 Routine
        routine_out = self.routine_agent.act(user_inputs.get("wake_time","06:30"), user_inputs.get("sleep_time","22:30"), user_inputs.get("commitments"))

        # 4 Tools analysis
        time_report = self.tools["time_opt"].analyze(user_inputs.get("waste_breakdown", {"phone":30, "idle":20}))
        life_score = self.tools["life_score"].compute(user_inputs.get("sleep_hours",7), user_inputs.get("exercise_mins",30), user_inputs.get("study_minutes",60), user_inputs.get("mood",6))

        # 5 Eval
        eval_out = self.eval_agent.act({"study_minutes": user_inputs.get("study_minutes",60), "exercise_mins":user_inputs.get("exercise_mins",30)})

        # 6 Loop summary
        loop_out = self.loop_agent.act()

        report = {
            "study": study_out,
            "fitness": fitness_out,
            "routine": routine_out,
            "time_tool": time_report,
            "life_score": life_score,
            "evaluation": eval_out,
            "daily_summary": loop_out,
            "memory_snapshot": json.loads(self.memory.export_json())
        }
        self.memory.append_history(self.user_id, {"pipeline_completed": True})
        return report

    def save_notebook_artifacts(self, filename="sla_memory.json"):
        # optional: save memory to a file in colab workspace
        with open(filename, "w") as f:
            f.write(self.memory.export_json())
        return filename

# instantiate orchestrator for demo use
orch = Orchestrator(user_id="vish_default")


In [16]:
# -------------------
# Demo: personalize these inputs and run
# -------------------
user_inputs = {
    "subject": "Digital Electronics",
    "exam_date": (date.today()+timedelta(days=18)).isoformat(),
    "daily_hours": 2,
    "height_cm": 170,
    "weight_kg": 56,
    "goal": "build_muscle",
    "wake_time": "06:30",
    "sleep_time": "23:00",
    "commitments": [("11:00","30m","online class")],
    "sleep_hours": 7.2,
    "exercise_mins": 45,
    "study_minutes": 110,
    "mood": 7,
    "waste_breakdown": {"phone_scroll":45, "idle":20}
}

print("Running pipeline... (this simulates sequential multi-agent run)\n")
report = orch.run_pipeline(user_inputs)

# Nicely print summary key parts (short)
print("=== Study Plan preview ===")
pprint(report["study"]["plan_preview"])

print("\n=== Fitness summary ===")
pprint({k: report["fitness"][k] for k in ["bmi","calories"]})

print("\n=== Routine blocks ===")
for b in report["routine"]["blocks"][:5]:
    print("-", b[0], ":", b[1])

print("\n=== Time optimizer ===")
pprint(report["time_tool"])

print("\n=== Life score ===")
print(report["life_score"])

print("\n=== Evaluation advice ===")
pprint(report["evaluation"])

# Save artifacts (optional)
saved = orch.save_notebook_artifacts("sla_memory_vish.json")
print(f"\nMemory exported to: {saved}")


Running pipeline... (this simulates sequential multi-agent run)

=== Study Plan preview ===
{'2025-11-25': {'hours': 2,
                'topics': ['Digital Electronics - Topic 1',
                           'Digital Electronics - Topic 2']},
 '2025-11-26': {'hours': 2,
                'topics': ['Digital Electronics - Topic 3',
                           'Digital Electronics - Topic 4']},
 '2025-11-27': {'hours': 2,
                'topics': ['Digital Electronics - Topic 5',
                           'Digital Electronics - Topic 6']},
 '2025-11-28': {'hours': 2,
                'topics': ['Digital Electronics - Topic 7',
                           'Digital Electronics - Topic 8']},
 '2025-11-29': {'hours': 2,
                'topics': ['Digital Electronics - Topic 9',
                           'Digital Electronics - Topic 10']},
 '2025-11-30': {'hours': 2,
                'topics': ['Digital Electronics - Topic 11',
                           'Digital Electronics - Topic 12']},
 '202

## Submission: Smart Life Assistant — Multi-Agent Capstone (Vivek Parmar)

**Abstract:**  
This notebook demonstrates a rule-based multi-agent Life Assistant that performs sequential planning and multi-step reasoning. It includes:
- **Multi-agent system:** StudyAgent, FitnessAgent, RoutineAgent, EvalAgent, plus a LoopAgent for daily checks.
- **Custom tools:** TimeOptimizerTool and LifeScoreTool (both implemented locally).
- **Memory / Session:** MemoryBank stores user profile and action history.
- **Sequential & Loop agents:** Agents are orchestrated sequentially and a daily loop agent simulates recurring checks.
- **Evaluation & metrics:** Simple life score and advice generation; memory export available.

**How to run:** Run all cells top-to-bottom. Edit `user_inputs` in the demo cell to personalize.  
**Why this is useful:** Demonstrates agent orchestration, tool usage, memory, evaluation — suitable for low-resource environments with no external APIs.

